In [1]:
import numpy as np
import json
import glob
import cv2
import pickle
from sklearn import metrics
from scipy.spatial import distance as dist

In [2]:
def get_camera_parameters(size):
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    
    return camera_matrix, dist_coeffs

In [3]:
def get_full_image_points(landmarks):
    image_points = np.zeros((68, 2))

    for i in range(68):
        image_points[i, :] = (landmarks[i]['x'], landmarks[i]['y'])
    
    return image_points

In [4]:
def get_full_model_points(filename='model_points.txt'):
    """Get all 68 3D model points from file"""
    raw_value = []
    with open(filename) as file:
        for line in file:
            raw_value.append(line)
    model_points = np.array(raw_value, dtype=np.float32)
    model_points = np.reshape(model_points, (3, -1)).T

    # Transform the model into a front view.
    model_points[:, 2] *= -1

    return model_points

In [5]:
def visualize_image(im, rotation_vector, translation_vector, image_points, camera_matrix, dist_coeffs,
                    iris_right, iris_left, label):
    for point in image_points:
        cv2.circle(im, (int(point[0]), int(point[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_left[0]), int(iris_left[1])), 3, (255, 0, 0), -1)
    cv2.circle(im, (int(iris_right[0]), int(iris_right[1])), 3, (255, 0, 0), -1)
    
    # Project the 3D point (0.55592, 6.5629, 300.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    (nose_end_point2D, jacobian) = cv2.projectPoints(
        np.array([(0.55592, 6.5629, 300.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
        )
    # Draw a line connecting the two points. This line must show
    # the direction out of the nose
    p1 = ( int(image_points[33][0]), int(image_points[33][1]) )
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
    cv2.line(im, p1, p2, (255,0,0), 2)
    
    # Display image
    cv2.imshow(label, im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean([eye[1]['x'], eye[1]['y']], [eye[5]['x'], eye[5]['y']])
    B = dist.euclidean([eye[2]['x'], eye[2]['y']], [eye[4]['x'], eye[4]['y']])
 
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean([eye[0]['x'], eye[0]['y']], [eye[3]['x'], eye[3]['y']])
 
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
 
    # return the eye aspect ratio
    return ear

In [7]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data)
# Convert python list to np array
keys_all = np.asarray(keys_all)
DATASET_SIZE = 2728
EAR_THRESHOLD = 0.2

model_points = get_full_model_points()

In [17]:
# for uuid in uuids[0:1]:
# # Remove the newline character
# uuid = uuid[:-1]
# print('UUID: ' + uuid)
uuid = '0e7047653b014264b33c1c3620e506f8'

with open('classifiers/rf/' + uuid + '.pickle', 'rb') as f:
    svm_classifier = pickle.load(f)
# with open('classifiers/svm/' + uuid +'.pickle', 'rb') as f:
#     svm_classifier = pickle.load(f)

# Array of keys for this uuid
keys = []
for i in range(DATASET_SIZE):
    key = keys_all[i]
    if(key.split('/')[0] == uuid):
        keys.append(key)
print('Number of examples for this uuid: {}'.format(len(keys)))

X = np.zeros((len(keys), 14, 1))
# X = [[]]
y = np.zeros(len(keys))

blinked_indices = []
valid_keys = []

for i in range(len(keys)):
    key = keys[i]
    im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
    size = im.shape

    landmarks = data[key]['landmarks']

    camera_matrix, dist_coeffs = get_camera_parameters(size)
    
    image_points = get_full_image_points(landmarks)

    # Solve the PnP problem with the parameters specified above
    # and obtain rotation and translation vectors
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_EPNP
        )

    # Data from Architecture #2. Reshape is done for compatibility reasons
    iris_right = np.reshape(np.asarray(data[key]['iris_right']), (2, 1))
    iris_left = np.reshape(np.asarray(data[key]['iris_left']), (2, 1))

    # Data from Architecture #3
    left_vector = np.asarray( (abs(iris_left[0] - landmarks[39]['x']), abs(iris_left[1] - landmarks[39]['y'])) )
    right_vector = np.asarray( (abs(iris_right[0] - landmarks[42]['x']), abs(iris_right[1] - landmarks[42]['y'])) )

    X[i, :] = np.concatenate((rotation_vector, translation_vector, iris_left, iris_right, 
                             left_vector, right_vector), axis=0)

#     iris_right = (data[key]['iris_right'])
#     iris_left = (data[key]['iris_left'])
#     left_vector = [abs(iris_left[0] - landmarks[39]['x']), abs(iris_left[1] - landmarks[39]['y'])]
#     right_vector = [abs(iris_right[0] - landmarks[42]['x']), abs(iris_right[1] - landmarks[42]['y'])]
#     X.append( rotation_vector.tolist() + translation_vector.tolist() + iris_left + iris_right + left_vector + right_vector )
    
    # Check if it is positive or negative example
    output = key.split('/')[1]
    if(output == 'positive'):
        y[i] = 1
    elif(output == 'negative'):
        y[i] = 0
    
    # Blink Detection
    leftEAR = eye_aspect_ratio(landmarks[36:42])
    rightEAR = eye_aspect_ratio(landmarks[42:48])
    ear = (leftEAR + rightEAR) / 2.0
    if(ear <= EAR_THRESHOLD):
        blinked_indices.append(i)
    else:
        valid_keys.append(key)


predicted_blinks = len(blinked_indices)
true_blinks = 0
for k in blinked_indices:
    if y[k] == 0:
        true_blinks += 1
        
print(predicted_blinks, true_blinks)

X_val = np.delete(X, blinked_indices, axis=0)
y_val = np.delete(y, blinked_indices, axis=0)

X_val = X_val.squeeze()

m = X_val.mean(axis=0)
std = X_val.std(axis=0)
X_scaled = (X_val-m)/std

y_pred = svm_classifier.predict(X_scaled)
classifier_accuracy = metrics.accuracy_score(y_val, y_pred)
accuracy = (classifier_accuracy*X_val.shape[0] + true_blinks)/(X_val.shape[0] + predicted_blinks)
print('Accuracy: {}'.format(accuracy))
print(metrics.confusion_matrix(y_val, y_pred))

Number of examples for this uuid: 47
18 18
Accuracy: 0.7446808510638298
[[ 4  1]
 [11 13]]


In [18]:
failed_indices = (y_val != y_pred)
print((failed_indices))

[False  True False False False  True False  True  True  True  True  True
 False  True False  True False False  True False False False False False
  True  True False False False]


In [19]:
for i in range(len(failed_indices)):
    if failed_indices[i]:
        key = valid_keys[i]
        label = key.split('/')[1]
        im = cv2.imread('dataset/' + key)
        size = im.shape
        
        landmarks = data[key]['landmarks']
        image_points = get_full_image_points(landmarks)
        camera_matrix, dist_coeffs = get_camera_parameters(size)
        
        visualize_image(im, X_val[i, 0:3], X_val[i, 3:6], image_points, camera_matrix, dist_coeffs, X_val[i, 6:8], X_val[i, 8:10], label)

In [83]:
print('False negatives:')
for i in range(len(failed_indices)):
    if y[i] == 1 and y_pred[i]==0:
        print(X[i, 10])
print('True negatives:')
for i in range(len(failed_indices)):
    if y[i] == 1 and y_pred[i]==1:
        print(X[i, 10])

False negatives:
2.8192101992093512
3.25477203956018
2.6060781478881836
2.746704725118775
0.3236083617577208
2.879357924828156
2.0970065410320444
2.22178151057318
1.7875318894019472
2.3511106050931403
2.700253706711976
2.539508196023803
2.940511263333832
0.9772604062006849
1.9619749509371331
2.6457295784583437
3.406033515930176
0.8055892724257205
4.744444443629334
1.307962564321656
4.023964184981139
2.4465326896080626
2.6731181511512148
2.45136345349826
0.5634184617262576
2.108732223510742
1.6958942413330078
2.09460126436673
0.5055019305302721
4.2833295968862615
0.24755804355328337
True negatives:
1.2932024002075195
1.4774847764235233
0.8276224869948123
0.8783782078669446
0.6298829592191169
2.037371378678529
1.7213633610651868
1.569537492898803
1.846873466785155
0.880313946650574


In [29]:
import time

In [39]:
start = time.perf_counter()
a = [[j for j in range(14)] for i in range(100000)]
end = time.perf_counter()

In [40]:
print(end - start)

0.1812627369999973
